In [1]:
import requests
import os
import re
import json
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from selenium import webdriver
import time
import matplotlib.pyplot as plt
import urllib
%matplotlib inline
from datetime import date

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import time


In [2]:

# Setup the ChromeDriver with a valid User-Agent
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # Optional, to run in headless mode
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# Set a valid User-Agent header
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
options.add_argument(f'user-agent={user_agent}')

driver = webdriver.Chrome(options=options)

headers = {
    'Accept-Language': 'en',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'X-FORWARDED-FOR': '2.21.184.0'
}

The chromedriver version (128.0.6613.137) detected in PATH at /opt/homebrew/bin/chromedriver might not be compatible with the detected chrome version (129.0.6668.100); currently, chromedriver 129.0.6668.100 is recommended for chrome 129.*, so it is advised to delete the driver in PATH and retry


In [3]:
pmg = 'https://pmg.org.za/question_replies/'

In [4]:
response = requests.get(pmg, headers=headers)
print(response.status_code)

200


In [5]:
# pmg_soup = BeautifulSoup(response.text, 'html.parser')

In [6]:
# def safe_extract(soup, selector, attribute, default=np.nan):
#         """Helper function to extract text from a BeautifulSoup tag safely."""
#         element = soup.find(selector, attribute)
#         return element.get_text(strip=True) if element else default


# # Helper function to extract text from multiple <p> tags within a specific div
# def safe_extract_multiple(soup, parent_selector, parent_attribute, child_selector='p'):
#     """Helper function to extract text from multiple <p> tags safely inside a parent container."""
#     parent_element = soup.find(parent_selector, parent_attribute)
#     if parent_element:
#         child_elements = parent_element.find_all(child_selector)
#         return [element.get_text(strip=True) for element in child_elements]
#     return []



In [7]:
def fetch_soup(pmg_url, headers):
    """
    Fetch the content of a URL (base URL + movie link) and return a BeautifulSoup object.

    Args:
        base_url (str): The base URL (e.g., 'https://www.imdb.com/').
        movie_link (str): The relative URL or path to the movie page.
        headers (dict): Headers to be used in the HTTP request.

    Returns:
        BeautifulSoup: Parsed HTML content of the page if successful, else None.
    """
    

    # Fetch the page content
    response = requests.get(pmg_url, headers=headers)
    
    # Check for a successful response and return BeautifulSoup object
    if response.status_code == 200:
        return BeautifulSoup(response.text, 'html.parser')
    
    return None


In [8]:
pmg_soup = fetch_soup('https://pmg.org.za/question_replies/9', headers)

In [9]:
# Helper function to extract text from a single tag
def safe_extract(soup, selector, attribute, default=np.nan):
    """Helper function to extract text from a BeautifulSoup tag safely."""
    element = soup.find(selector, attribute)
    return element.get_text(strip=True) if element else default


# Helper function to extract text from multiple <p> tags within a specific div
def safe_extract_multiple(soup, parent_selector, parent_attribute, child_selector='p'):
    """Helper function to extract text from multiple <p> tags safely inside a parent container."""
    parent_element = soup.find(parent_selector, parent_attribute)
    if parent_element:
        child_elements = parent_element.find_all(child_selector)
        return [element.get_text(strip=True) for element in child_elements]
    return []

In [10]:
# Extract the required fields
pmg_date = safe_extract(pmg_soup, 'h4', {}, default=np.nan)  # Extracts the date and NW552 reference
pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-heading'}, default=np.nan)  # Extract the name and credentials of the person asking
# pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-body'}, default=np.nan)  # Extract the name and credentials of the person asking
pm_question = safe_extract(pmg_soup, 'p', {}, default=np.nan)  # Extract the question text

# Extract the content from all <p> tags within the specific 'committee-question-reply' div
pmg_answer_paragraphs = safe_extract_multiple(pmg_soup, 'div', {'class': 'committee-question-reply'})

# Join paragraphs into one string for better readability
pmg_answer = " ".join(pmg_answer_paragraphs)



# Output the extracted values
print("Date:", pmg_date)
print("\nCredentials:", pmg_credentials)
print("\nQuestion:", pm_question)
print("\nAnswer:", pmg_answer)

Date: 13 September 2024 -CW141

Credentials: Ngcobo, Ms ZNto ask the
Minister of Police:

Question: (1) What mechanisms has the SA Police Service (SAPS) implemented for ongoing monitoring and evaluation of the promotion policy to ensure it meets its objectives and adapts to the constantly changing needs of South Africa (details furnished);
(2) whether the system undermines effective management by prioritising rank over qualifications, skills and experience that are needed for different roles; if not, what is the position in this regard; if so, what are the relevant details;
(3) how often should the promotion structures be revised to allow properly qualified support staff to progress to management roles within the SAPS, while allowing operational personnel to focus on policing crime?

Answer: Find replyhere


In [11]:
def get_pmg_content(pmg_soup):


    # Extract the required fields from 'committee-question-detail'
    pmg_date = safe_extract(pmg_soup, 'h4', {}, default=np.nan)  # Extract the date and NW552 reference from the <h4> tag
    
    # pmg_credentials = safe_extract(pmg_soup, 'div', {'class': 'media-body'}, default=np.nan)  # Extract name and credentials of the person asking from 'media-body' class
    pmg_credentials = safe_extract(pmg_soup, 'h4', {'class': 'media-heading'}, default=np.nan)

    pm_question = safe_extract(pmg_soup, 'p', {}, default=np.nan)  # Extract the question from the first <p> tag in the 'committee-question-detail' div
    # Extract the content from all <p> tags within the 'committee-question-reply' div
    pmg_answer_paragraphs = safe_extract_multiple(pmg_soup, 'div', {'class': 'committee-question-reply'})

    # Join paragraphs into one string for better readability
    pmg_answer = " ".join(pmg_answer_paragraphs)

    return pmg_date, pmg_credentials, pm_question, pmg_answer

In [12]:

def get_all_pmg_content(pages=5, headers=headers, verbose=False):
    columns = ['date', 'mp', 'question', 'answer']
    data = []
    done = 0
    base_url = 'https://pmg.org.za/question_replies/'

    start = time.time()
    
    for page in range(1, pages):
        
        soup = fetch_soup(f'{base_url}{page}', headers)
        
        if soup:
            pages_stat = list(get_pmg_content(soup))
            # data.append([page] + pages_stat)
            data.append(pages_stat)
        else:
            # data.append([page] + [np.nan] * 4)
            data.append([np.nan] * 4)
        
        if verbose:
            done += 1
            print(f'Done with {done} pages. {pages - done} left. Elapsed time: {(time.time() - start):.1f} seconds')
        
        time.sleep(1)  # To avoid overwhelming the server
    
    return pd.DataFrame(data, columns=columns)
                  


In [13]:

def get_all_pmg_content(pages=5, headers=None, verbose=False):
    columns = ['date', 'mp', 'question', 'answer']
    data = []
    done = 0
    base_url = 'https://pmg.org.za/question_replies/'

    start = time.time()

    # Step 1: Scrape the base page first (without page number)
    soup = fetch_soup(base_url, headers)
    if soup:
        # Find all 'committee-question-detail' elements on the base page
        question_details = soup.select('article > section .committee-question-detail')
        
        # For each 'committee-question-detail' element, scrape the content
        for detail in question_details:
            content = get_pmg_content(detail)  # Extract the content for each detail
            data.append(content)  # Append the extracted content to the data list
    else:
        # If no content is found on the base page, append NaNs
        data.append([np.nan] * 4)

    # Step 2: Loop through additional pages (1, 2, ..., n)
    for page in range(1, pages + 1):  # Loop through specified numbered pages
        soup = fetch_soup(f'{base_url}{page}', headers)  # Fetch soup for each numbered page

        if soup:
            # Find all 'committee-question-detail' elements on the numbered pages
            question_details = soup.select('article > section .committee-question-detail')
            
            # For each 'committee-question-detail' element, scrape the content
            for detail in question_details:
                content = get_pmg_content(detail)  # Extract the content for each detail
                data.append(content)  # Append the extracted content to the data list
        else:
            # If no content is found, append NaNs to maintain consistent rows
            data.append([np.nan] * 4)

        # Print verbose progress
        if verbose:
            done += 1
            print(f'Done with {done} pages. {pages - done} left. Elapsed time: {(time.time() - start):.1f} seconds')

        time.sleep(1)  # Pause to avoid overwhelming the server

    # Return the scraped data as a DataFrame
    return pd.DataFrame(data, columns=columns)


In [15]:
_sample_pmg = get_all_pmg_content(pages=5, headers=headers, verbose=1)

Done with 1 pages. 4 left. Elapsed time: 1.2 seconds
Done with 2 pages. 3 left. Elapsed time: 3.0 seconds
Done with 3 pages. 2 left. Elapsed time: 5.2 seconds
Done with 4 pages. 1 left. Elapsed time: 6.9 seconds
Done with 5 pages. 0 left. Elapsed time: 8.5 seconds


In [16]:
_sample_pmg.head()

,date,mp,question,answer
0,09 October 2024 -NW177,"Pambo, Mr Vto ask the\nMinister in the Presidency",What (a) total number of members of the State ...,The Agency has one member and two accomplices ...
1,09 October 2024 -NW12,"Myburgh, Mr NGto ask the\nMinister in the Pres...","Whether, in the wake of alleged widespread abu...",The State Security Agency as the country’s civ...
2,09 October 2024 -NW978,"Madlala, Mr KBto ask the\nMinister of Water an...",Noting that her department is one of the most ...,The Department has a responsibility to ensure ...
3,09 October 2024 -NW612,"Bodlani, Ms Tto ask the\nMinister in the Presi...",What (a) total amount in costs did the Governm...,"R3 496 673,08; REPLY (b): The budget line item..."
4,09 October 2024 -NO10,"Mathafa, Mr OMto ask the\nMinister of Justice ...",Considering that incidents of racism amongst y...,


In [17]:
_sample_pmg['mp'][1]

'Myburgh, Mr NGto ask the\nMinister in the Presidency'

In [18]:
# Step 1: Clean up the `date` column to remove anything after the date
_sample_pmg['date'] = _sample_pmg['date'].str.extract(r'^([0-9]{2} [A-Za-z]+ [0-9]{4})')
# Step 2: Convert the cleaned date column to datetime format
_sample_pmg['date'] = pd.to_datetime(_sample_pmg['date'], format='%d %B %Y')
# Step 3: Split the 'mp' column using '\n' and get the first part as 'mp' and last part as 'responder'
_sample_pmg['mp'], _sample_pmg['responder'] = _sample_pmg['mp'].str.split('\n').str[0], _sample_pmg['mp'].str.split('\n').str[1]
# Step 4: Split the 'mp' column using '\n' and get the first part as 'mp' and last part as 'responder'
_sample_pmg['mp'] = _sample_pmg['mp'].str.replace('to ask the', '', regex=False)

_sample_pmg.head()

,date,mp,question,answer,responder
0,2024-10-09,"Pambo, Mr V",What (a) total number of members of the State ...,The Agency has one member and two accomplices ...,Minister in the Presidency
1,2024-10-09,"Myburgh, Mr NG","Whether, in the wake of alleged widespread abu...",The State Security Agency as the country’s civ...,Minister in the Presidency
2,2024-10-09,"Madlala, Mr KB",Noting that her department is one of the most ...,The Department has a responsibility to ensure ...,Minister of Water and Sanitation
3,2024-10-09,"Bodlani, Ms T",What (a) total amount in costs did the Governm...,"R3 496 673,08; REPLY (b): The budget line item...",Minister in the Presidency
4,2024-10-09,"Mathafa, Mr OM",Considering that incidents of racism amongst y...,,Minister of Justice and Constitutional Develop...


In [20]:
_sample_pmg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       300 non-null    datetime64[ns]
 1   mp         300 non-null    object        
 2   question   300 non-null    object        
 3   answer     300 non-null    object        
 4   responder  300 non-null    object        
dtypes: datetime64[ns](1), object(4)
memory usage: 11.8+ KB


In [21]:
# Convert the dictionary into a DataFrame
# _2024_films_links_df = pd.DataFrame(list(_2024_films_links.items()), columns=['Title', 'Link'])

# Write the DataFrame to a Parquet file with Brotli compression
_sample_pmg.to_parquet('_sample_pmg.parquet.brotli', compression='brotli')

print("Data written to parquet successfully.")


Data written to parquet successfully.
